## Example script to generate .json configuration file required for GA optimization.

In [3]:
import os
import pandas as pd

### GA hyperparameters

In [4]:
n_organisms = 256 #number of organisms
n_elites = 16 #number of elite organisms
n_generations = 2 #number of generations

config = {'n_organisms': n_organisms,
          'n_elites': n_elites,
          'n_generations': n_generations,
          
          'crossover_rate': 1.0,
          'mutation_rate': 1.0,
          'gamma': 0.05,
         }

In [5]:
output_folder_name = 'OUTPUT'
output_folder_path = '../../results/'

config['output_folder_name'] = os.path.join(output_folder_path, output_folder_name)

## Fitness function

Fitness function is declared in ga/mpi_scripts/loss_utils.py . Currently, either RMSE of the signal or the weighted superposition of the RMSE of the signal and its derivative is supported (RMSE, RMSE_GRAD correspondingly). 

In [6]:
config['loss'] = 'RMSE'

## Input data

Folders containg CSV data files of voltage-clamp protocol and recorded patch-clamp traces are indicated below. Protocol folder is supposed to contain .csv file with the protocol itself and the .cfg file with the same name where the holding potential between sweeps is saved after import from .abf file. 

In [7]:
trace_dir = '../data/traces/' #patch-clamp recorded trace
protocol_dir = '../data/protocols/' # voltage clamp protocol

In [8]:
### Traces
traces = pd.DataFrame(os.listdir(trace_dir), columns=['filename'])
traces.index.name ='n_trace'
traces

,filename
n_trace,
0,activation.csv
1,NEW_NAME.csv
2,inactivation.csv


In [9]:
### Protocols

protocols = pd.DataFrame(os.listdir(protocol_dir), columns=['filename'])
protocols.index.name ='n_protocol'
protocols

,filename
n_protocol,
0,activation.csv
1,NEW_NAME.csv
2,inactivation.csv


In [10]:
# Input data directories contents is shown above. Particular prtocol or trace is chosen by number.

# number of trace
n_trace = 1
#number of protocol
n_protocol = 1

trace_path = os.path.abspath(os.path.join(trace_dir, traces.filename[n_trace]))
protocol_path = os.path.abspath(os.path.join(protocol_dir, protocols.filename[n_protocol]))


individual = {}
individual['filename_phenotype'] = trace_path
individual['filename_protocol'] = protocol_path

## Model
The model of experimental setup should be compiled as a 'filename_so' C library. Model state variables, algebraic variables and constants are described in 'filename_legend_states', 'filename_legend_algebraic' and 'filename_legend_constants' .csv files correspondingly. The lists of variables and constants is stored as a SUNDIALS NVector internally, with the order corresponding to the respective csv files. The name of output current state variable is controlled by 'columns_model' parameter of the .json configuraration file.

In [11]:
dirname_model = '../src/model_ctypes/PC_model/'
filename_so = 'libina.so'

config['filename_so'] =  os.path.abspath(os.path.join(dirname_model, filename_so))
config['filename_legend_states'] =  os.path.abspath(os.path.join(dirname_model, 'legend_states.csv'))
config['filename_legend_algebraic'] =  os.path.abspath(os.path.join(dirname_model, 'legend_algebraic.csv'))
config['filename_legend_constants'] =  os.path.abspath(os.path.join(dirname_model, 'legend_constants.csv'))

config['columns_model'] = ['I_out']

In [12]:
config

{'n_organisms': 256,
 'n_elites': 16,
 'n_generations': 2,
 'crossover_rate': 1.0,
 'mutation_rate': 1.0,
 'gamma': 0.05,
 'output_folder_name': '../../results/OUTPUT',
 'loss': 'RMSE',
 'filename_so': '/home/ras/Projects/Veronika/code_article/GIT2/PCoptim/src/model_ctypes/PC_model/libina.so',
 'filename_legend_states': '/home/ras/Projects/Veronika/code_article/GIT2/PCoptim/src/model_ctypes/PC_model/legend_states.csv',
 'filename_legend_algebraic': '/home/ras/Projects/Veronika/code_article/GIT2/PCoptim/src/model_ctypes/PC_model/legend_algebraic.csv',
 'filename_legend_constants': '/home/ras/Projects/Veronika/code_article/GIT2/PCoptim/src/model_ctypes/PC_model/legend_constants.csv',
 'columns_model': ['I_out']}

## Free parameters and their boundaries

In [13]:
config['experimental_conditions'] = {}

Free parameters are divided in two parts: experimental setup parameters designated as 'individual' parameters, and ionic current parameters designated as 'individual' parameters. Both lists are expected to be not empty. Absoulte scale for the parameter and its boundaries is used if 'is_multiplier' boolean is set to False. Otherwise parameter value is considered to be a scaler of original model value, logarithmic scale is used in this case. 

NOTE: Simultaneous optimization of multiple experimental traces is supported. In this case 'common' parameters are supposed to be the same in every experiment, while 'individual' are supposed to depend on particular recording. Because of that the recorded trace and the protocol filenames are stored as 'individual' parameters as well. Change 'individual' to 'individual#' in this case, where # corresponds to the number of the trace.

In [14]:
individual['params'] = {'c_p': {'bounds': [0.01,5.0], 'is_multiplier': True},
                   'c_m': {'bounds': [0.1, 5.0],  'is_multiplier': True},
                   'R': {'bounds': [0.1, 100.0], 'is_multiplier': True},
                   'R_f': {'bounds': [0.01, 50.0], 'is_multiplier': True},
                   'g_max': {'bounds': [0.05, 100.0], 'is_multiplier': True},
                   'g_leak': {'bounds': [0.1, 10.0], 'is_multiplier': True},
                   'x_c_comp': {'bounds': [0.1, 100.0], 'is_multiplier': True},
                   'x_r_comp': {'bounds': [0.1, 10.0], 'is_multiplier': True},
                   'alpha': {'bounds': [0.72, 0.78], 'is_multiplier': False},
                   'v_rev': {'bounds': [16.0, 80.0], 'is_multiplier': False},
                   'v_off': {'bounds': [-2.0, 2.0], 'is_multiplier': False},
                   'tau_z': {'bounds': [0.4, 6.0], 'is_multiplier': True}}

In [15]:
default_settings = {'bounds': [0.1, 10], 'is_multiplier': True}

common = {}
common_parameters = ['a0_m','b0_m','delta_m','s_m', 'tau_m_const',
                    'a0_h', 'b0_h','delta_h','s_h', 'tau_h_const',
                    'a0_j', 'b0_j', 'delta_j', 's_j', 'tau_j_const',
                    ]


# Using default settings
common['params'] = {param: default_settings for param in common_parameters}

# You can change bounds for free parameters if it's necessary
common['params']['tau_m_const']['bounds'] =  [0.05, 2.0]



In [16]:
common['params']['v_half_h'] = {'bounds': [60.0, 80.0], 'is_multiplier': False}
common['params']['v_half_m'] = {'bounds': [20.0, 30.0], 'is_multiplier': False}
common['params']['k_h'] = {'bounds': [6.0, 15.0], 'is_multiplier': False}
common['params']['k_m'] = {'bounds': [6.0, 15.0], 'is_multiplier': False}


In [17]:
config['experimental_conditions']['common'] = common
config['experimental_conditions']['individual'] = individual

In [18]:
config

{'n_organisms': 256,
 'n_elites': 16,
 'n_generations': 2,
 'crossover_rate': 1.0,
 'mutation_rate': 1.0,
 'gamma': 0.05,
 'output_folder_name': '../../results/OUTPUT',
 'loss': 'RMSE',
 'filename_so': '/home/ras/Projects/Veronika/code_article/GIT2/PCoptim/src/model_ctypes/PC_model/libina.so',
 'filename_legend_states': '/home/ras/Projects/Veronika/code_article/GIT2/PCoptim/src/model_ctypes/PC_model/legend_states.csv',
 'filename_legend_algebraic': '/home/ras/Projects/Veronika/code_article/GIT2/PCoptim/src/model_ctypes/PC_model/legend_algebraic.csv',
 'filename_legend_constants': '/home/ras/Projects/Veronika/code_article/GIT2/PCoptim/src/model_ctypes/PC_model/legend_constants.csv',
 'columns_model': ['I_out'],
 'experimental_conditions': {'common': {'params': {'a0_m': {'bounds': [0.05,
      2.0],
     'is_multiplier': True},
    'b0_m': {'bounds': [0.05, 2.0], 'is_multiplier': True},
    'delta_m': {'bounds': [0.05, 2.0], 'is_multiplier': True},
    's_m': {'bounds': [0.05, 2.0], 'is_

## Save config

In [19]:
import json

config_name = 'NEW_NAME'
with open(f"../ga/configs/{config_name}.json", 'w') as f:
    f.write(json.dumps(config, indent=4))